In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import os
print(os.listdir("../input"))

['Housing Price data set.csv']


**GRADIENT DESCENT METHOD**

In [2]:
df = pd.read_csv('../input/Housing Price data set.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 13 columns):
Unnamed: 0    546 non-null int64
price         546 non-null float64
lotsize       546 non-null int64
bedrooms      546 non-null int64
bathrms       546 non-null int64
stories       546 non-null int64
driveway      546 non-null object
recroom       546 non-null object
fullbase      546 non-null object
gashw         546 non-null object
airco         546 non-null object
garagepl      546 non-null int64
prefarea      546 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 55.5+ KB


In [4]:
df = df.replace({'yes': 1, 'no': 0})
df.head()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000.0,5850,3,1,2,1,0,1,0,0,1,0
1,2,38500.0,4000,2,1,1,1,0,0,0,0,0,0
2,3,49500.0,3060,3,1,1,1,0,0,0,0,0,0
3,4,60500.0,6650,3,1,2,1,1,0,0,0,0,0
4,5,61000.0,6360,2,1,1,1,0,0,0,0,0,0


In [5]:
df.columns


Index(['Unnamed: 0', 'price', 'lotsize', 'bedrooms', 'bathrms', 'stories',
       'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl',
       'prefarea'],
      dtype='object')

In [6]:
x_features = ['lotsize', 'bedrooms', 'bathrms', 'stories','driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl',
       'prefarea']

In [7]:
x_df = df[x_features]
x_df.head()
y_df = df['price']
X_train, X_test, Y_train, Y_test = tts(x_df, y_df, test_size = 0.3, random_state = 5)

In [8]:
m = len(Y_train)
alpha = 0.01
lambd = 5
num_iters = 800
weights = np.zeros((12,1))
X_train = (X_train - np.mean(X_train))/np.std(X_train)
X_test = (X_test - np.mean(X_test))/np.std(X_test)
Y_train = Y_train[:,np.newaxis]
Y_test = Y_test[:,np.newaxis]

In [9]:
X_train = X_train.assign(b=1)
X_test = X_test.assign(b=1)
# X_train.head()
X_test.head()
# Y_train
# Y_test

,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea,b
288,-0.800418,-1.470358,-0.551121,-1.023486,0.393611,-0.511408,-0.769567,-0.177332,-0.633805,0.434920,-0.633805,1
526,-0.035629,-0.084503,1.457410,2.267723,0.393611,-0.511408,-0.769567,-0.177332,1.577771,1.623700,-0.633805,1
496,-0.313346,-0.084503,-0.551121,-1.023486,0.393611,-0.511408,1.299432,-0.177332,-0.633805,-0.753861,-0.633805,1
483,-0.526974,-1.470358,-0.551121,-1.023486,0.393611,-0.511408,-0.769567,-0.177332,-0.633805,-0.753861,1.577771,1
409,0.583892,-0.084503,1.457410,1.170654,0.393611,-0.511408,-0.769567,-0.177332,1.577771,-0.753861,1.577771,1


In [10]:
def gradientDescentMulti(X, y, weights, alpha, iterations):
    m = len(y)
    print(X.shape)
    print(y.shape)
    print(weights.shape)
    for _ in range(iterations):
        temp = np.dot(X, weights) - y
        temp = np.dot(X.T, temp)
        weights = weights*(1-(alpha*lambd)/m) - (alpha/m) * temp
    return weights

In [11]:
weights = gradientDescentMulti(X_train, Y_train, weights, alpha, num_iters)

(382, 12)
(382, 1)
(12, 1)


In [12]:
weights.shape

(12, 1)

In [13]:
def GD_predict(X, weights):
    return np.dot(X, weights)

In [14]:
weights

array([[ 8016.36016718],
       [ 2162.41029291],
       [ 7242.57780858],
       [ 5100.93697542],
       [ 2374.12060385],
       [ 1518.16463832],
       [ 2766.809421  ],
       [ 2728.50565789],
       [ 5568.34225005],
       [ 3845.04007499],
       [ 4360.51805024],
       [67172.10393676]])

In [15]:
GD_predict(X_test.iloc[0], weights)

array([41288.17436079])

In [16]:
Y_predicted = [GD_predict(x, weights) for x in X_test.values]
mean_squared_error(Y_test, Y_predicted)

154312436.62637576

In [17]:
Y_test[0]

array([30000.])

**NORMAL EQUATION METHOD**

In [18]:
df = df.assign(b=1)
df.columns

Index(['Unnamed: 0', 'price', 'lotsize', 'bedrooms', 'bathrms', 'stories',
       'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl',
       'prefarea', 'b'],
      dtype='object')

In [19]:
x_features = ['b','lotsize', 'bedrooms', 'bathrms', 'stories']

In [20]:
x_df = df[x_features]
x_df.head()
y_df = df['price']
X_train, X_test, Y_train, Y_test = tts(x_df, y_df, test_size = 0.3, random_state = 5)

In [21]:
X_train.head()

,b,lotsize,bedrooms,bathrms,stories
122,1,4400,2,1,1
9,1,5500,3,2,4
297,1,3120,3,1,2
203,1,3180,4,2,2
509,1,4040,3,1,2


In [22]:
Y_train.head()

122    37000.0
9      88500.0
297    50000.0
203    55500.0
509    64000.0
Name: price, dtype: float64

In [23]:
def normal_train(X, Y):
    lambd = 5
    transposeX = np.transpose(X)
    N = X.shape[1]
    I = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i==j and i!=0:
                I[i,j] = 1
            
    try:
        A = np.linalg.inv(np.dot(transposeX,X)+(lambd*I))
        B = np.dot(transposeX,Y)
        return np.dot(A,B)
    except numpy.linalg.LinAlgError:
        print("X is not invertible")
def normal_predict(X, weights):
    transposeW = np.transpose(weights)
    return np.dot(transposeW, X)

In [24]:
weights = normal_train(X_train, Y_train)

In [25]:
weights = weights.astype(float)
X_test = X_test.astype(float)

In [26]:
normal_predict(weights, X_test.iloc[0])

43560.61293609085

In [27]:
Y_test.iloc[0]

30000.0

In [28]:
Y_predicted = [normal_predict(weights, x) for x in X_test.values]
mean_squared_error(Y_test, Y_predicted)

254579984.33444157